In [ ]:
# =========================================================== #
#   Lesson: Working with Google Earth Engine
# 
# =========================================================== #

In [1]:
# Import the working tools
import ee 
import geemap
import ipywidgets as widgets
from IPython.display import display

In [6]:
# Initialize GEE
ee.Initialize(project='learngee-471509')

In [7]:
# Create an interactive map
Map = geemap.Map(
    center=[25.2048, 55.2708],  # Dubai coordinates
    zoom=10,
    height='600px'
)

In [ ]:
# Get a cloud-free Sentinel-2 image
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(oman) \
    .filterDate('2024-01-01', '2024-03-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .select(['B4', 'B3', 'B2', 'B8', 'B11']) \
    .median()

# Calculate NDVI
ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Print information about the processed images
print("Successfully processed Sentinel-2 data for Oman")
print("Available bands:", sentinel2.bandNames().getInfo())
print("Area bounds:", oman.buffer(50000).bounds().getInfo())

# Get NDVI statistics
ndvi_stats = ndvi.reduceRegion(
    reducer=ee.Reducer.minMax().combine(ee.Reducer.mean(), sharedInputs=True),
    geometry=oman.buffer(50000),
    scale=10,
    maxPixels=1e9
)
print("📈 NDVI Statistics:", ndvi_stats.getInfo())

# Alternative 1: Export to Google Drive
def export_to_drive():
    # Export true color image
    task1 = ee.batch.Export.image.toDrive(
        image=sentinel2.select(['B4', 'B3', 'B2']),
        description='Oman_TrueColor',
        scale=10,
        region=oman.buffer(25000),
        fileFormat='GeoTIFF'
    )
    
    # Export NDVI
    task2 = ee.batch.Export.image.toDrive(
        image=ndvi,
        description='Oman_NDVI',
        scale=10,
        region=oman.buffer(25000),
        fileFormat='GeoTIFF'
    )
    
    # Start tasks
    task1.start()
    task2.start()
    
    print("🚀 Export tasks started. Check your Google Drive in a few minutes.")
    print(f"📝 Task 1 ID: {task1.id}")
    print(f"📝 Task 2 ID: {task2.id}")

# Alternative 2: Generate static map
def generate_static_map():
    # Create a static map URL
    url = sentinel2.select(['B4', 'B3', 'B2']).getThumbURL({
        'min': 0,
        'max': 3000,
        'dimensions': 1024,
        'region': oman.buffer(50000),
        'format': 'png'
    })
    print("🖼️ Static map URL (copy to browser):")
    print(url)

# Run alternatives
print("\n" + "="*50)
print("Choose an option:")
print("1. Export to Google Drive: export_to_drive()")
print("2. Generate static map URL: generate_static_map()")
print("="*50)

In [ ]:
# Uncomment to use:
# export_to_drive()
generate_static_map()

✅ Successfully processed Sentinel-2 data for Oman
📊 Available bands: ['B4', 'B3', 'B2', 'B8', 'B11']
🗺️ Area bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[58.10525317719209, 23.136129024928714], [59.08067221127958, 23.136129024928714], [59.08067221127958, 24.0358432975741], [58.10525317719209, 24.0358432975741], [58.10525317719209, 23.136129024928714]]]}
📈 NDVI Statistics: {'NDVI_max': 1, 'NDVI_mean': -0.09665263658279283, 'NDVI_min': -1}

Choose an option:
1. Export to Google Drive: export_to_drive()
2. Generate static map URL: generate_static_map()
🖼️ Static map URL (copy to browser):
https://earthengine.googleapis.com/v1/projects/learngee-471509/thumbnails/f50e73785c4c7daabc6b7b650707f6fb-9ea3d1fed9d3ac517f8f3812cd21bdba:getPixels
